<a href="https://colab.research.google.com/github/AlbertFlorinus/housing_analysis/blob/master/final_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Regression using PCA Analysis**

## **Introduction**

<p>The amount of variables contained within housing dataset are of varying degree of importance for predicted sales price. While intuition would lead one to believe that the area of the property contributes more to the sales price than for example the number of fireplaces, manual selection feature selection is prone to bias and even impractical. This lab explores the use and advantage of Principal Component analysis (PCA) for selecting features to perform regression on. </p>

## **Description of PCA**
<p>
PCA is an unsupervised algorithm which calculates which features in a dataset best describe the data points overall.
 
This does NOT mean that the features PCA ranks highest are the most useful for describing the class attribute of each datapoint. It does tend to eliminate redundant features. 

The score tells how a variance in a feature will affect the other features, a higher score will bring a higher effect.

PCA is useful because it allows one to only use a subset of the dataset but still be able to keep a lot of information about the relations between the features.
</p>

## **Research Question**
### Does regressions modeled after PCA-chosen attributes outperform regressions modeled after randomly-chosen attributes?


## Data Analysis
<p>

</p>

### Imports
<p>For this lab we will be using several Python libraries, including: <br>
Matplotlib, for interactive and noninteractive graphs. <br>
Pandas, for dealing with large sets of data.<br>
Numpy, for creating regression models and associated objects. <br>
SciPy, for generating correlation coefficients. <br>
sklearn, for their PCA algorithm. <br>
iPyWidgets, for adding interactivity to the graphs.
</p>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from scipy import stats
from sklearn.decomposition import PCA
import random

import ipywidgets as widgets
from ipywidgets import interact, interactive, Dropdown

In [ ]:
!git clone https://github.com/AlbertFlorinus/housing_analysis

Cloning into 'housing_analysis'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 247 (delta 125), reused 173 (delta 56), pack-reused 0
Receiving objects: 100% (247/247), 6.11 MiB | 19.02 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [ ]:
#preprocessing and import
df = pd.read_csv('/content/housing_analysis/house-data/train.csv')
df = df[ df.select_dtypes(include=np.number).columns.tolist()]
df.dropna(inplace=True)

In [ ]:
def pca_dec(data, n):
  pca = PCA(n)
  X_dec = pca.fit_transform(data)
  return X_dec, pca

#Decomposing the train set:
pca_train_results, pca_train = pca_dec(df, 10)

#Creating a table with the explained variance ratio
names_pcas = [f"PCA Component {i}" for i in range(1, 11, 1)]
scree = pd.DataFrame(list(zip(names_pcas, pca_train.explained_variance_ratio_)), columns=["Component", "Explained Variance Ratio"])

df_new = pd.DataFrame({'PCA':pca_train.components_[0], 'Variable Names':list(df.columns)})
df_new = df_new.sort_values('PCA', ascending=False)

In [ ]:
#Sorting the absolute values of the first principal component by magnitude
df2 = pd.DataFrame(df_new)
df2['PCA']=df2['PCA'].apply(np.absolute)
df2 = df2.sort_values('PCA', ascending=False)
df_new.head()

,PCA,Variable Names
37,0.999535,SalePrice
3,0.029625,LotArea
16,0.004449,GrLivArea
12,0.003306,TotalBsmtSF
13,0.002830,1stFlrSF


In [ ]:
def statistic(data_reg,predicted, measured, attribute):
    """
    For higher order regressions, determines whether to use kendall or spearman
    """
    mono = True
    der = np.polyder(data_reg)
    roots = np.roots(der)
            
    for root in roots:
        if 1 < root < df[attribute].max():
            mono = False
        if mono:
            return [data_reg,predicted,stats.spearmanr(predicted, measured)[0], attribute]
        else:
            return [data_reg,predicted,stats.kendalltau(predicted, measured)[0],attribute]

In [ ]:
def regress(degree, attribute):
    """
    performs regression on given attribute with degree
    """
    measured = df["SalePrice"]
    model = {}
    xp = np.linspace(1, df[attribute].max()+df[attribute].mean()/4, 20)
    
    data_reg = np.polyfit(df[attribute], df["SalePrice"], degree)
    predicted = np.polyval(data_reg, df[attribute].tolist())
    
    if degree == 1:
        #use pearson
        model[attribute] = [data_reg, predicted, stats.pearsonr(predicted, measured)[0], attribute]
    
    elif degree > 1:
        #use function statistic to determine type of correlation coefficient
        model[attribute] = statistic(data_reg,predicted, measured, attribute)

    return model

In [ ]:
def mean_correlation(feature_collection:dict):
    """
    helper function for factor_test
    """
    n = len(feature_collection)
    avg_correlation = sum( np.array( list(feature_collection.values()) )[:,1] )/n
    return avg_correlation

In [ ]:
def factor_test(degree:int, features:list):
    """
    performs single variable regression on all inputed features,
    returns the mean correlation
    """
    feature_collection = {i:[degree, regress(degree, i)[i][2]] for i in features}
    return mean_correlation(feature_collection)

In [ ]:
#list of features ordered by PCA rank, first element(salesprice removed)
PCA_features = df_new["Variable Names"].tolist()[1:]

In [ ]:
def n_taken(rank):
    """
    rank is the degree for the regression
    """
    n_used = [i for i in range(1,len(df.keys())+1)]

    n_corr = []

    for n in n_used:
        n_corr.append( factor_test(rank, PCA_features[:n]) )
    
    n_ran_used = [i for i in range(1, len(df.keys())+1)]
    n_ran_corr = []

    for n in n_ran_used:
        count = 0
        m = 50
        for i in range(m):
            #take n random features m times
            features = random.sample(list(df.keys()), n)
            count += factor_test(rank, features)

        avg_c = count/m
        n_ran_corr.append(avg_c)
    
    return [[n_used, n_corr],[n_ran_used, n_ran_corr]]

In [ ]:
models = {1: n_taken(1), 2: n_taken(2), 3: n_taken(3)}

In [ ]:
#interactive plotter
attribute = Dropdown(options = df.keys())
@interact(degree = (1, 6), attribute = attribute)
def g(degree, attribute):
    measured = df["SalePrice"]
    COLORS = ["red", "blue", "green", "orange", "yellow", "gray", "cyan", "purple"]
    model = {}
    xp = np.linspace(1, df[attribute].max()+df[attribute].mean()/4, 20)
    for i in range(1, degree+1):
        data_reg = np.polyfit(df[attribute], df["SalePrice"], i)
        predicted = np.polyval(data_reg, df[attribute].tolist())
        if i == 1:
            model[i] = [data_reg,
                             predicted,
                             stats.pearsonr(predicted, measured)[0]]
        
        
        elif i > 1:
            model[i] = statistic(data_reg, predicted, measured, attribute )

    fig = plt.figure(figsize=(14,9))
    ax = plt.axes()
    plt.scatter(df[attribute], df["SalePrice"], label = f"degree : correlation", color="black")
    for i in range(1, degree+1):
        ax.plot(xp, np.polyval(model[i][0], xp), label = f"{i} : {round(model[i][2], 5)}", color = COLORS[i-1] )
    
    ax.set_ylim(bottom=0, top = df["SalePrice"].max()+df["SalePrice"].mean()/4)
    ax.set_xlim(0, df[attribute].max()+df[attribute].mean()/4)
    ax.set_title("")
    ax.set_title(f"SalePrice vs {attribute}")
    ax.set_xlabel(f"{attribute}")
    ax.set_ylabel("SalePrice")
    ax.grid()
    ax.legend()

interactive(children=(IntSlider(value=3, description='degree', max=6, min=1), Dropdown(description='attribute'…

In [ ]:
@interact(degree = (1, 3))
def pcm(degree):
    fig = plt.figure(figsize=(9,5))
    ax = fig.add_subplot()

    ax.plot(models[degree][0][0],models[degree][0][1],color="green", label="mean_correlation(n)")
    ax.plot(models[degree][1][0],models[degree][1][1], color="blue", label = "mean_correlation(n_random)")
    ax.set_xlabel("n features")
    ax.set_ylabel("Mean correlation")
    ax.set_title("Mean correlation of lin reg by amount of PCA features used")
    #renders legends in the plot
    ax.legend() 
    ax.grid()

interactive(children=(IntSlider(value=2, description='degree', max=3, min=1), Output()), _dom_classes=('widget…

## Conclusion
<p>
As seen in the figure above, featurs highly ranked by PCA coincide with a higher mean correlation coefficent when compared to randomly chosen features. Raising the amount of PCA features lowers the mean correlation coefficient, a result of including less predictive features. The large increase in mean correlation coefficient for PCA features up to n=4 can be explained in part by the highest ranked feature, LotArea performing poorly for predicting SalesPrice. Randomly chosen features consistently underperforms PCA ranked features, and only gets matching performance when the number of features approaches all features available. This confirms the hypothesis that features selected by PCA rank tend to outperform random features for modelling a regression.
</p>